# Tutorial 7: Plotting and selecting data from netCDF

This tutorial will continue working with netCDF files in Xarray and we'll move beyond the basics. In the previous tutorial, we looked at the entire area of the netCDF file. But, as scientists, we often have a specific region of interest that is a subset of the larger dataset region. In this tutorial, we'll learn how to select and plot only an area of interest to us. 

By the end of this tutorial, you will be able to:
* align a cartopy region with netCDF data
* create a map using continuous or gridded data from netCDF
* select a point and a region of interest from the entire dataset

<font color=red>**Note:**</font> This tutorial uses the cmoceans packages, which is a library of very nice colormaps for plotting. You can check out more info here https://matplotlib.org/cmocean/#. 

In [ ]:
# for data manipulation and opening files
import xarray as xr
import pandas as pd

# for plotting
import matplotlib.pyplot as plt
from cartopy import crs, feature
import numpy as np
import cmocean

### Downloading data

We will try out a new type of gridded data for plotting.

1. Go to https://www.ncei.noaa.gov/access/world-ocean-atlas-2018/. 
2. Play around with the different data available. There are several ocean data products. For this tutorial, we will be using sea surface temperature. The code examples will be specific to this data, but you can choose any data product and just change the variable names.    
3. Once you have selected a data product from the left hand menu, click "netCDF" under available data formats. Choose a time period of interest. This tutorial uses the 2005-2017 summer period. Click "Update data links" at the bottom of the left hand menu.
    - **Reminder:** If you choose to work with a different data product, year, or grid size, you will need to change the file link below and the variable names. 
4. On the right hand side, select the Annual data link, t15_01.nc. This will take you to the THREDDS Server page.
5. Click the OPeNDAP URL option (top link in the list). 
6. Copy the Data URL on the OPeNDAP page. 
7. Paste the URL below in the `file = 'https://` line. Change any other variables. 

In [ ]:
# change this file name and/or path if you are using different data
file = 'https://www.ncei.noaa.gov/thredds-ocean/dodsC/ncei/woa/temperature/A5B7/1.00/woa18_A5B7_t15_01.nc'

In [ ]:
# use xarray to open the file you chose
data = xr.open_dataset(file,engine='netcdf4',decode_times=False)
# decode_times = False needs to be set here because of the way time is coded in this file
# what happens if you remove decode_times = False?

In [ ]:
# what does this data look like?
data

### Plotting the entire area

When we first open data, it can sometimes be useful to plot the entire area. This may give us an idea of what the data looks like or any areas of interest.

In [ ]:
# the data of interest
temp = data['t_an']

In [ ]:
# look at it just in case
temp

In [ ]:
# make a basic contour plot using the lons, lats, and then the surface level of the temperature
plt.contourf(temp['lon'],temp['lat'],temp[0,0,:,:])
plt.show()

This is a pretty bad figure, but we can get some basic information from it. First, everything seems really patchy. Why is that?

**Knowledge Check:** Why are there so many empty spaces in the figure above? What does this have to do with the type of data we are using?

Ocean data is only interested in water. So any land surface areas will not have data. The white spaces in the figure above are from the land - you can see the outlines of the continents!

### Selecting areas on a map

Now that we've looked at the entire dataset area, we can zoom in on a particular section. There are a few ways to do this, but we'll practice the easiest first.

The easiest method to zoom into an area on a map is to only plot that area with cartopy. This is fast and useful for making a map for presentations, but there are some downsides.   
* First, this method does not remove the rest of the data, it just hides it from our view. If you wanted to find the average chlorophyll-a concentration in the zoomed in area, you could not calculate it from the map because the rest of the data actually still exists.    
* Second, outliers in the data outside the zoomed in area might affect the colorbar scale of the displayed map. So you could have a map area with concentrations all below 20 mg/m$^{3}$, but a larger concentration elsewhere in the dataset will skew the colorbar scale and you won't be able to see the low concentrations clearly.   
* Finally, for really big datasets, it might take a while to plot. You're still technically plotting all the data, it just isn't displayed in the figure area. So if the dataset is really big, you're wasting time plotting data that you don't actually see. 

In [ ]:
# use a specialty colormap for algae
cmap = cmocean.cm.thermal

In [ ]:
# select lat and lon region of interest
# use the map created above to start zooming in on an area that has a lot of purple coverage
# the area of purple coverage might be different than the lats and lons here if you chose a different date
latmin = 37
latmax = 45
lonmin = -65
lonmax = -75

In [ ]:
# set the extents of our box
extent = [lonmin,lonmax,latmin,latmax]

In [ ]:
# select map projection - do you remember what this is for this dataset?
proj = crs.PlateCarree()

In [ ]:
# in order to plot a subset of the data on the map, we must make a grid of the coordinates
# first, get a list of each coordinate
lats = temp['lat']
lons = temp['lon']

# then make a grid
XX,YY = np.meshgrid(lons,lats)

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.STATES)
ax.add_feature(feature.RIVERS)
im = plt.contourf(XX,YY,temp[0,0,:,:],cmap=cmap,transform=proj)
plt.colorbar(im,orientation='horizontal',label='Temperature (˚C)')
plt.show()

What do you notice about this figure? What about the colorbar scale? What could make this figure better?

The figure above has a colorbar scale that goes from -5 to 40 ˚C, even though the data in the figure does not seem to go this high or low. This is one of the problems with this plotting method. Your colorbar scale may be different if you used a different date of data.

Instead of plotting all the data but hiding most of it, we can instead select a subsection of the data using slicing. This method takes a few more steps because you need to know the area of interest. However, there are several benefits such as being able to really focus on the actual area of interest. 

In [ ]:
# identify the map area extent
latmin = 37
latmax = 45
lonmin = -65
lonmax = -75

In [ ]:
# slice the data using these limits
subset = temp.sel(lat=slice(latmin,latmax),lon=slice(lonmax,lonmin))

In [ ]:
# check out the data and the change in dimensions
subset

In [ ]:
# set the extents of our box
extent = [lonmin,lonmax,latmin,latmax]
# select map projection
proj = crs.PlateCarree()

In [ ]:
# in order to plot a subset of the data on the map, we must make a grid of the coordinates
# first, get a list of each coordinate
lats = subset['lat']
lons = subset['lon']

# then make a grid
XX,YY = np.meshgrid(lons,lats)

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.STATES)
ax.add_feature(feature.RIVERS)
im = plt.contourf(XX,YY,subset[0,0,:,:],cmap=cmap,transform=proj)
plt.colorbar(im,orientation='horizontal',label='Temperature (˚C)')
plt.show()

The colorbar scale on this figure is a little better, and the code cell also ran a lot faster. But why is there so much white space between land and the ocean?

**Knowledge Check:** Why is there empty space between the land and ocean, and why does the ocean end in straight lines? Hint: you might look back at the World Ocean Atlas website where you downloaded the data.

The empty spaces are there because we downloaded data on a 1˚ by 1˚ grid. This is quite a large grid, and it will miss all of the small coastal details. 

### Different grids and data types

Let's download a different dataset from the World Ocean Atlas on a different grid and make some comparisons.

1. Go to https://www.ncei.noaa.gov/access/world-ocean-atlas-2018/. 
2. Play around with the different data available. There are several ocean data products. The following code uses annual average density on a quarter degree (1/4˚) grid.    
3. Once you have selected a data product from the left hand menu, click "netCDF" under available data formats and click "Update data links" at the bottom of the left hand menu.
    - **Reminder:** If you choose to work with a different data product, year, or grid size, you will need to change the file link below and the variable names. 
4. On the right hand side, select the Annual data link. This will take you to the THREDDS Server page.
5. Click the OPeNDAP URL option (top link in the list). 
6. Copy the Data URL on the OPeNDAP page. 
7. Paste the URL below in the `file = 'https://` line. Change any other variables. 

In [ ]:
file2 = 'https://www.ncei.noaa.gov/thredds-ocean/dodsC/ncei/woa/density/decav/0.25/woa18_decav_I00_04.nc'

In [ ]:
data2 = xr.open_dataset(file2,engine='netcdf4',decode_times=False)

In [ ]:
# look at the data. What differences do you notice?
data2

The lat and lon dims are much larger now because they have 4x as many grid points!

Let's plot the same zoomed in area as before.

In [ ]:
# the data of interest
dens = data2['I_an'][0,0,:,:]
dens

In [ ]:
# select lat and lon region of interest
latmin = 37
latmax = 45
lonmin = -65
lonmax = -75

In [ ]:
# the subset area of interest
subset = dens.sel(lat=slice(latmin,latmax),lon=slice(lonmax,lonmin))

In [ ]:
# set the extents of our box
extent = [lonmin,lonmax,latmin,latmax]

In [ ]:
# select map projection - do you remember what this is for this dataset?
proj = crs.PlateCarree()

In [ ]:
# in order to plot a subset of the data on the map, we must make a grid of the coordinates
# first, get a list of each coordinate
lats = subset['lat']
lons = subset['lon']

# then make a grid
XX,YY = np.meshgrid(lons,lats)

In [ ]:
cmap2 = cmocean.cm.dense

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent(extent)
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.STATES)
ax.add_feature(feature.RIVERS)
im = plt.contourf(XX,YY,subset,cmap=cmap2,transform=proj)
plt.colorbar(im,orientation='horizontal',label='Density $(kg/m^{3})$')
plt.show()

The data coverage close to the coast is much better now that we have a smaller grid size. Let's zoom back out and look at the entire ocean.

In [ ]:
lats = dens['lat']
lons = dens['lon']

XX,YY = np.meshgrid(lons,lats)

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent([-180,180,-90,90])
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.LAND,color='grey',alpha=0.3)
ax.add_feature(feature.RIVERS)
im = plt.contourf(XX,YY,dens,cmap=cmap2,transform=proj)
plt.colorbar(im,orientation='horizontal',label='Density $(kg/m^{3})$')
plt.show()

Is there anything in particular you notice about this figure? Maybe the colorbar scale? In fact, the colorbar scale is a little worrying. Density should not be $0 kg/m^3$, that implies there is nothing there. Let's check the min value of our dataset.

In [ ]:
dens.min()

At least the minimum isn't actually zero, but it is really small. This highlights the importance of knowing your data. These density values we've been using are not absolute densities, they are sigma-t densities. They are the difference from $1000 kg/m^{3}$, which is why we can have such small values. In reality, the density values should be 1000 + dens.

In [ ]:
# create a pyplot figure
fig = plt.figure(figsize=(8,10))
# create a new axes instance with the map information
ax = fig.add_subplot(1,1,1,projection=proj)
ax.set_extent([-180,180,-90,90])
# add gridlines
gl = ax.gridlines(crs.PlateCarree(),draw_labels=True,linewidth=1,color='gray',alpha=0.5,linestyle='--')
# add features
ax.add_feature(feature.LAND,color='grey',alpha=0.3)
ax.add_feature(feature.RIVERS)
im = plt.contourf(XX,YY,dens+1000,cmap=cmap2,transform=proj)
plt.colorbar(im,orientation='horizontal',label='Density $(kg/m^{3})$')
plt.show()

These are the actual density values, not the sigma values.

### Exercises

**1.** Download a new set of data and plot a coastal region of your choosing.   
a) Download the data and look at it.

b) Based on your plot, where is the important coastal data located? Zoom in on this area by creating new lat and lon extents and subselecting the coastal data.

In [ ]:
latmin =
latmax =
lonmin =
lonmax =

In [ ]:
coastal_subset = 

c) Plot this new data.